<a href="https://colab.research.google.com/github/carlos-alves-one/-ML-Zoomcamp-Week-4/blob/main/ML_Zoomcamp_Week_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Goldsmiths University of London
**Author....: Carlos Manuel de Oliveira Alves**<br>
**Student..: cdeol003**<br>
**Created..: 27/09/2022**

In [1]:
# Import libraries for the project
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Import the library warnings to ignore the warnings
import warnings
warnings.filterwarnings("ignore")